In [1]:
# Basic imports
import os
import sys
import time
import pickle
from itertools import product
import warnings

# System path modification
sys.path.insert(0, '..')

# Data handling
import pandas as pd
import numpy as np

# Machine learning imports
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.linear_model import (
    LinearRegression, Lasso, LassoCV, MultiTaskLasso, MultiTaskLassoCV,
    ElasticNet, ElasticNetCV, MultiTaskElasticNet, MultiTaskElasticNetCV
)
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score

from sklearn.cross_decomposition import PLSRegression
from sklearn.inspection import permutation_importance

# Custom modules
from src.train import *
from src.functions import *
from src.plots import *
from src.dataset import *
from src.multixgboost import *
from src.wrapper import *
from src.debug import *

# Visualizatiokn 
import matplotlib.pyplot as plt
import seaborn as sns

# Deep learning and machine learning specific 
import torch
from pytorch_tabnet.tab_model import TabNetRegressor
import xgboost as xgb
import shap

from pytorch_tabular.config import DataConfig, TrainerConfig, OptimizerConfig
from pytorch_tabular.models.common.heads import LinearHeadConfig

from pytorch_tabular.models import (
    GatedAdditiveTreeEnsembleConfig,
    DANetConfig,
    TabTransformerConfig,
    FTTransformerConfig,
    TabNetModelConfig,
)

# Ignore warnings
warnings.filterwarnings("ignore")

# Print CUDA availability for PyTorch
print(torch.cuda.is_available())
print(torch.cuda.device_count())

from omegaconf import DictConfig
torch.serialization.safe_globals([DictConfig])

False
0


## Load data 

In [2]:
data = load_pickle_data_palettes()

results_pickle_folder = "../pickle/"

# Unpack data
df_X, df_y, df_all, df_FinalCombination = data["df_X"], data["df_y"], data["df_all"], data["df_FinalCombination"]
dict_select = data["dict_select"]

# Unpack colormaps
full_palette, gender_palette, dx_palette = data["colormaps"].values()

# Train-Test Split

In [3]:
idx_train = list(df_X.isna().any(axis=1))
idx_test = list(~df_X.isna().any(axis=1))

set_intersect_rid = set(df_all[idx_train].RID).intersection(set(df_all[idx_test].RID))
intersect_rid_idx = df_all.RID.isin(set_intersect_rid)

for i, bool_test in enumerate(idx_test): 
    if intersect_rid_idx.iloc[i] & bool_test:
        idx_test[i] = False
        idx_train[i] = True
        
df_X[["APOE_epsilon2", "APOE_epsilon3", "APOE_epsilon4"]] = df_X[["APOE_epsilon2", "APOE_epsilon3", "APOE_epsilon4"]].astype("category")

df_X_train = df_X.loc[idx_train]
df_X_test = df_X.loc[idx_test]

df_y_train = df_y.loc[idx_train]
df_y_test = df_y.loc[idx_test]

c_train = df_all[["AGE", "PTGENDER", "PTEDUCAT"]].iloc[idx_train]
c_test = df_all[["AGE", "PTGENDER", "PTEDUCAT"]].iloc[idx_test]

In [4]:
df_all.SubjectID.iloc[idx_test]

3609    128_S_2002
5631    116_S_4167
5662    033_S_4176
5780    098_S_4215
5950    018_S_4349
6069    941_S_4292
6077    116_S_4453
6085    135_S_4489
6224    033_S_4505
6400    014_S_4576
6429    073_S_4300
7021    003_S_2374
7192    033_S_4179
Name: SubjectID, dtype: object

Define all the models and combinations to try out with their hyperparameters. 

In [65]:
random_state=42

# Continuous Imputer List (list of tuples with unique strings and corresponding instances)
continuous_imputer_list = [
    ("KNNImputer", KNNImputer(n_neighbors=1)),
]

# Ordinal Imputer List (list of tuples with unique strings and corresponding instances)
ordinal_imputer_list = [
    ("SimpleImputer_most_frequent", SimpleImputer(strategy="most_frequent")),
]

# Predictive Models List (list of tuples with unique strings and corresponding instances)
predictive_models_list = [
    ("LinearRegression", LinearRegression()),
    ("MultiTaskElasticNet", MultiTaskElasticNet()),
    ("MultiTaskElasticNet_tuned", MultiTaskElasticNet(**{'alpha': 0.01, 'l1_ratio': 0.01})),
    ("MultiTaskLasso", MultiTaskLasso()),
    ("MultiTaskLasso_tuned", MultiTaskLasso(**{'alpha': 0.001})),
    ("RandomForestRegressor", RandomForestRegressor()),
    ("XGBoostRegressor", XGBoostRegressor()),
    ("XGBoostRegressor_tuned", XGBoostRegressor(**{'colsample_bytree': 0.5079831261101071, 'learning_rate': 0.0769592094304232, 'max_depth': 6, 'min_child_weight': 7, 'subsample': 0.8049983288913105})),
    ("TabNetRegressor_default", TabNetModelWrapper(n_a=8, n_d=8)),
    ("TabNetRegressor_custom", TabNetModelWrapper(n_a=32, n_d=32)),
    ("PLSRegression_4_components", PLSRegression(n_components=4))
]

In [66]:
ordinal_features = ['APOE_epsilon2', 'APOE_epsilon3', 'APOE_epsilon4']
continuous_features = [col for col in df_X_train.columns if col not in ordinal_features]

# Prepare Tabular configurations (shared for all PyTorch models)
data_config = DataConfig(
    target=df_y_train.columns.tolist(),
    continuous_cols=continuous_features,
    categorical_cols=ordinal_features
)
trainer_config = TrainerConfig(
    batch_size=1024, max_epochs=10, auto_lr_find=False,
    early_stopping="valid_loss", early_stopping_mode="min", early_stopping_patience=5,
    checkpoints="valid_loss", load_best=True, progress_bar="nones",
)
optimizer_config = OptimizerConfig()
head_config = LinearHeadConfig(dropout=0.1).__dict__

predictive_models_list += [
    ("GatedAdditiveTreeEnsembleConfig_tab", 
    TabularModelWrapper(
        GatedAdditiveTreeEnsembleConfig(
        task="regression",
        head="LinearHead",
        head_config=head_config,
        gflu_stages=6,
        gflu_dropout=0.0,
        tree_depth=5,
        num_trees=20,
        chain_trees=False,
        share_head_weights=True), data_config, trainer_config, optimizer_config 
    )),
    ("DANetConfig_tab",
    TabularModelWrapper(
        DANetConfig(
        task="regression",
        head="LinearHead",
        head_config=head_config,
        n_layers=8,
        k=5,
        dropout_rate=0.1), data_config, trainer_config, optimizer_config
    )),
    ("TabTransformerConfig_tab",
        TabularModelWrapper(
        TabTransformerConfig(
        task="regression",
        head="LinearHead",
        head_config=head_config,
        embedding_initialization="kaiming_uniform",
        embedding_bias=False), data_config, trainer_config, optimizer_config
    )),
    ("TabNetModelConfig_tab",
        TabularModelWrapper(
        TabNetModelConfig(
        task="regression",
        head="LinearHead",
        head_config=head_config,
        n_d=8,
        n_a=8,
        n_steps=3,
        gamma=1.3,
        n_independent=2,
        n_shared=2), data_config, trainer_config, optimizer_config
    )),
]

In [67]:
# Generate all combinations
combinations = list(product(continuous_imputer_list, ordinal_imputer_list, predictive_models_list))

# Display all combinations
for continuous_imputer, ordinal_imputer, model in combinations:
    print(f"Continuous Imputer: {continuous_imputer[0]}, Ordinal Imputer: {ordinal_imputer[0]}, Model: {model[0]}")

print(f"Combinations of preprocessing and models to test : {len(combinations)}")

Continuous Imputer: KNNImputer, Ordinal Imputer: SimpleImputer_most_frequent, Model: LinearRegression
Continuous Imputer: KNNImputer, Ordinal Imputer: SimpleImputer_most_frequent, Model: MultiTaskElasticNet
Continuous Imputer: KNNImputer, Ordinal Imputer: SimpleImputer_most_frequent, Model: MultiTaskElasticNet_tuned
Continuous Imputer: KNNImputer, Ordinal Imputer: SimpleImputer_most_frequent, Model: MultiTaskLasso
Continuous Imputer: KNNImputer, Ordinal Imputer: SimpleImputer_most_frequent, Model: MultiTaskLasso_tuned
Continuous Imputer: KNNImputer, Ordinal Imputer: SimpleImputer_most_frequent, Model: RandomForestRegressor
Continuous Imputer: KNNImputer, Ordinal Imputer: SimpleImputer_most_frequent, Model: XGBoostRegressor
Continuous Imputer: KNNImputer, Ordinal Imputer: SimpleImputer_most_frequent, Model: XGBoostRegressor_tuned
Continuous Imputer: KNNImputer, Ordinal Imputer: SimpleImputer_most_frequent, Model: TabNetRegressor_default
Continuous Imputer: KNNImputer, Ordinal Imputer: S

In [68]:
# Initialize HDF5 file
results_file = '../pickle/training_2_dict_results.pickle'

if os.path.exists(results_file): 

    with open(results_file, "rb") as input_file:
        all_dict_results = pickle.load(input_file)

else : 
    all_dict_results = []

In [69]:
for result in all_dict_results:
    print(result["params"])

{'ordinal_imputer': 'SimpleImputer_most_frequent', 'continuous_imputer': 'KNNImputer', 'model': 'LinearRegression', 'train_shape': (2881, 348), 'test_shape': (13, 348)}
{'ordinal_imputer': 'SimpleImputer_most_frequent', 'continuous_imputer': 'KNNImputer', 'model': 'MultiTaskElasticNet', 'train_shape': (2881, 348), 'test_shape': (13, 348)}
{'ordinal_imputer': 'SimpleImputer_most_frequent', 'continuous_imputer': 'KNNImputer', 'model': 'MultiTaskElasticNet_tuned', 'train_shape': (2881, 348), 'test_shape': (13, 348)}
{'ordinal_imputer': 'SimpleImputer_most_frequent', 'continuous_imputer': 'KNNImputer', 'model': 'MultiTaskLasso', 'train_shape': (2881, 348), 'test_shape': (13, 348)}
{'ordinal_imputer': 'SimpleImputer_most_frequent', 'continuous_imputer': 'KNNImputer', 'model': 'MultiTaskLasso_tuned', 'train_shape': (2881, 348), 'test_shape': (13, 348)}
{'ordinal_imputer': 'SimpleImputer_most_frequent', 'continuous_imputer': 'KNNImputer', 'model': 'RandomForestRegressor', 'train_shape': (2881

In [ ]:
if False : 
    params_comb = [{'ordinal_imputer': 'SimpleImputer_most_frequent', 'continuous_imputer': 'KNNImputer', 'model': 'GatedAdditiveTreeEnsembleConfig_tab', 'train_shape': [2893, 348], 'test_shape': [1, 348]},
    {'ordinal_imputer': 'SimpleImputer_most_frequent', 'continuous_imputer': 'KNNImputer', 'model': 'DANetConfig_tab', 'train_shape': [2893, 348], 'test_shape': [1, 348]},
    {'ordinal_imputer': 'SimpleImputer_most_frequent', 'continuous_imputer': 'KNNImputer', 'model': 'TabTransformerConfig_tab', 'train_shape': [2893, 348], 'test_shape': [1, 348]},
    {'ordinal_imputer': 'SimpleImputer_most_frequent', 'continuous_imputer': 'KNNImputer', 'model': 'TabNetModelConfig_tab', 'train_shape': [2893, 348], 'test_shape': [1, 348]},
    {'ordinal_imputer': 'NoImputer', 'continuous_imputer': 'NoImputer', 'model': 'GatedAdditiveTreeEnsembleConfig_tab', 'train_shape': [2893, 348], 'test_shape': [1, 348]},
    {'ordinal_imputer': 'NoImputer', 'continuous_imputer': 'NoImputer', 'model': 'DANetConfig_tab', 'train_shape': [2893, 348], 'test_shape': [1, 348]},
    {'ordinal_imputer': 'NoImputer', 'continuous_imputer': 'NoImputer', 'model': 'TabTransformerConfig_tab', 'train_shape': [2893, 348], 'test_shape': [1, 348]},
    {'ordinal_imputer': 'NoImputer', 'continuous_imputer': 'NoImputer', 'model': 'TabNetModelConfig_tab', 'train_shape': [2893, 348], 'test_shape': [1, 348]}]

    for params in params_comb:
        all_dict_results = clean_dict_list(all_dict_results, remove_if_none=False, remove_key_val={"params": params})

In [71]:
all_dict_results = clean_dict_list(all_dict_results, remove_if_none=False, remove_key_val={'fitting_time':None})

Removed due to key-value match: fitting_time=None
Removed due to key-value match: fitting_time=None


In [73]:
for continuous_imputer, ordinal_imputer, model in combinations:
    name_continuous_imputer, continuous_imputer_instance = continuous_imputer
    name_ordinal_imputer, ordinal_imputer_instance = ordinal_imputer
    name_model, model_instance = model

    params = {
        "ordinal_imputer": name_ordinal_imputer, 
        "continuous_imputer": name_continuous_imputer, 
        "model": name_model, "train_shape" : df_X_train.shape, 
        "test_shape": df_X_test.shape
    }
    print(f"Training :{name_model}")

    if any(result['params'] == params for result in all_dict_results):
        # Skip this iteration if the combination exists
        print(f"Skipping existing combination: {params.values()}")
        
        continue

    try: 
    
        # Now you can call your `train_model` function with these components
        dict_results = train_imputer_model(
            df_X_train, df_X_test, df_y_train, df_y_test,
            c_train, c_test,
            ordinal_imputer_instance, name_ordinal_imputer,
            continuous_imputer_instance, name_continuous_imputer,
            model_instance, name_model,
            separate_imputers=True  # Or however you want to specify
        )

    except Exception as e:  

        print(e)
    
        dict_results = {
        "params": params, 
        "imputation_time": None,
        "fitting_time": None, 
        "results_adj": None, 
        "results_org": None
    }
        
    # Optionally keep the all_dict_results list updated
    all_dict_results.append(dict_results)

        # Save the updated results back to the pickle file
    with open(results_file, 'wb') as f:
        pickle.dump(all_dict_results, f)


Training :LinearRegression
Skipping existing combination: dict_values(['SimpleImputer_most_frequent', 'KNNImputer', 'LinearRegression', (2881, 348), (13, 348)])
Training :MultiTaskElasticNet
Skipping existing combination: dict_values(['SimpleImputer_most_frequent', 'KNNImputer', 'MultiTaskElasticNet', (2881, 348), (13, 348)])
Training :MultiTaskElasticNet_tuned
Skipping existing combination: dict_values(['SimpleImputer_most_frequent', 'KNNImputer', 'MultiTaskElasticNet_tuned', (2881, 348), (13, 348)])
Training :MultiTaskLasso
Skipping existing combination: dict_values(['SimpleImputer_most_frequent', 'KNNImputer', 'MultiTaskLasso', (2881, 348), (13, 348)])
Training :MultiTaskLasso_tuned
Skipping existing combination: dict_values(['SimpleImputer_most_frequent', 'KNNImputer', 'MultiTaskLasso_tuned', (2881, 348), (13, 348)])
Training :RandomForestRegressor
Skipping existing combination: dict_values(['SimpleImputer_most_frequent', 'KNNImputer', 'RandomForestRegressor', (2881, 348), (13, 348

In [74]:
# Store data (serialize)
with open(results_file, 'wb') as handle:
    pickle.dump(all_dict_results, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [75]:
with open('../pickle/training_2_dict_results.pickle', "rb") as input_file:
    dict_results_split = pickle.load(input_file)

In [76]:
dict_results_split

[{'params': {'ordinal_imputer': 'SimpleImputer_most_frequent',
   'continuous_imputer': 'KNNImputer',
   'model': 'LinearRegression',
   'train_shape': (2881, 348),
   'test_shape': (13, 348)},
  'imputation_time': 3.2466392517089844,
  'fitting_time': 0.22939062118530273,
  'results_adj': {'mse_score': array([0.65983176, 0.48974311, 0.40951464, 0.56684282]),
   'mae_score': array([0.63965764, 0.55144938, 0.56535002, 0.59999667]),
   'r2': array([0.33312259, 0.44426866, 0.144955  , 0.31048062]),
   'explained_variance': array([0.36730884, 0.47760179, 0.14545957, 0.43985495]),
   'corr': array([0.60633336, 0.69220668, 0.4020632 , 0.66605104])},
  'results_org': {'mse_score': array([0.65983176, 0.4897431 , 0.40951466, 0.56684279]),
   'mae_score': array([0.63965764, 0.55144938, 0.56535003, 0.59999666]),
   'r2': array([0.29810904, 0.45135258, 0.20239561, 0.33725209]),
   'explained_variance': array([0.33409019, 0.48426081, 0.20286628, 0.46160331]),
   'corr': array([0.57814397, 0.6981620

# Train models only on MRI features to compare performances

## Test train split

In [77]:
idx_train = list(df_X.isna().any(axis=1))
idx_test = list(~df_X.isna().any(axis=1))

set_intersect_rid = set(df_all[idx_train].RID).intersection(set(df_all[idx_test].RID))
intersect_rid_idx = df_all.RID.isin(set_intersect_rid)

for i, bool_test in enumerate(idx_test): 
    if intersect_rid_idx.iloc[i] & bool_test:
        idx_test[i] = False
        idx_train[i] = True

In [78]:
df_X[["APOE_epsilon2", "APOE_epsilon3", "APOE_epsilon4"]] = df_X[["APOE_epsilon2", "APOE_epsilon3", "APOE_epsilon4"]].astype("category")

In [79]:
df_X_train = df_X[dict_select["MRIth"]].loc[idx_train]
df_X_test = df_X[dict_select["MRIth"]].loc[idx_test]

df_y_train = df_y.loc[idx_train]
df_y_test = df_y.loc[idx_test]

c_train = df_all[["AGE", "PTGENDER", "PTEDUCAT"]].iloc[idx_train]
c_test = df_all[["AGE", "PTGENDER", "PTEDUCAT"]].iloc[idx_test]

In [80]:
random_state=42
n_imputation_iter = 10

# Continuous Imputer List (list of tuples with unique strings and corresponding instances)
continuous_imputer_list = [
    ("NoImputer", KNNImputer(n_neighbors=1)),

]

# Ordinal Imputer List (list of tuples with unique strings and corresponding instances)
ordinal_imputer_list = [
    ("NoImputer", SimpleImputer(strategy="most_frequent")),
]

# Predictive Models List (list of tuples with unique strings and corresponding instances)
predictive_models_list = [
    ("LinearRegression", LinearRegression()),
    ("MultiTaskElasticNet", MultiTaskElasticNet()),
    ("MultiTaskElasticNet_tuned", MultiTaskElasticNet(**{'alpha': 0.01, 'l1_ratio': 0.01})),
    ("MultiTaskLasso", MultiTaskLasso()),
    ("MultiTaskLasso_tuned", MultiTaskLasso(**{'alpha': 0.001})),
    ("RandomForestRegressor", RandomForestRegressor()),
    ("XGBoostRegressor", XGBoostRegressor()),
    ("XGBoostRegressor_tuned", XGBoostRegressor(**{'colsample_bytree': 0.5079831261101071, 'learning_rate': 0.0769592094304232, 'max_depth': 6, 'min_child_weight': 7, 'subsample': 0.8049983288913105})),
    ("TabNetRegressor_default", TabNetModelWrapper(n_a=8, n_d=8)),
    ("TabNetRegressor_custom", TabNetModelWrapper(n_a=32, n_d=32)),
    ("PLSRegression_4_components", PLSRegression(n_components=4))
]

In [81]:
ordinal_features = ['APOE_epsilon2', 'APOE_epsilon3', 'APOE_epsilon4']
continuous_features = [col for col in df_X_train.columns if col not in ordinal_features]

# Prepare Tabular configurations (shared for all PyTorch models)
data_config = DataConfig(
    target=df_y_train.columns.tolist(),
    continuous_cols=continuous_features,
    categorical_cols=[]
)
trainer_config = TrainerConfig(
    batch_size=1024, max_epochs=10, auto_lr_find=False,
    early_stopping="valid_loss", early_stopping_mode="min", early_stopping_patience=5,
    checkpoints="valid_loss", load_best=True, progress_bar="nones",
)
optimizer_config = OptimizerConfig()
head_config = LinearHeadConfig(dropout=0.1).__dict__

predictive_models_list += [
    ("GatedAdditiveTreeEnsembleConfig_tab", 
    TabularModelWrapper(
        GatedAdditiveTreeEnsembleConfig(
        task="regression",
        head="LinearHead",
        head_config=head_config,
        gflu_stages=6,
        gflu_dropout=0.0,
        tree_depth=5,
        num_trees=20,
        chain_trees=False,
        share_head_weights=True), data_config, trainer_config, optimizer_config 
    )),
    ("DANetConfig_tab",
    TabularModelWrapper(
        DANetConfig(
        task="regression",
        head="LinearHead",
        head_config=head_config,
        n_layers=8,
        k=5,
        dropout_rate=0.1), data_config, trainer_config, optimizer_config
    )),
    ("TabTransformerConfig_tab",
        TabularModelWrapper(
        TabTransformerConfig(
        task="regression",
        head="LinearHead",
        head_config=head_config,
        embedding_initialization="kaiming_uniform",
        embedding_bias=False), data_config, trainer_config, optimizer_config
    )),
    ("TabNetModelConfig_tab",
        TabularModelWrapper(
        TabNetModelConfig(
        task="regression",
        head="LinearHead",
        head_config=head_config,
        n_d=8,
        n_a=8,
        n_steps=3,
        gamma=1.3,
        n_independent=2,
        n_shared=2), data_config, trainer_config, optimizer_config
    )),
]

In [82]:
# Generate all combinations
combinations = list(product(continuous_imputer_list, ordinal_imputer_list, predictive_models_list))

# Display all combinations
for continuous_imputer, ordinal_imputer, model in combinations:
    print(f"Continuous Imputer: {continuous_imputer[0]}, Ordinal Imputer: {ordinal_imputer[0]}, Model: {model[0]}")

print(f"Combinations of preprocessing and models to test : {len(combinations)}")

Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: LinearRegression
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: MultiTaskElasticNet
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: MultiTaskElasticNet_tuned
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: MultiTaskLasso
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: MultiTaskLasso_tuned
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: RandomForestRegressor
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: XGBoostRegressor
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: XGBoostRegressor_tuned
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: TabNetRegressor_default
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: TabNetRegressor_custom
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: PLSRegression_4_components
Continuous Imputer: NoImputer, Ordinal 

In [83]:
# Initialize HDF5 file
results_file = '../pickle/training_2_dict_results.pickle'

with open(results_file, "rb") as input_file:
    all_dict_results = pickle.load(input_file)

In [84]:
for continuous_imputer, ordinal_imputer, model in combinations:
    name_continuous_imputer, continuous_imputer_instance = continuous_imputer
    name_ordinal_imputer, ordinal_imputer_instance = ordinal_imputer
    name_model, model_instance = model

    try: 
    
        # Now you can call your `train_model` function with these components
        dict_results = train_imputer_model(
            df_X_train, df_X_test, df_y_train, df_y_test,
            c_train, c_test,
            ordinal_imputer_instance, name_ordinal_imputer,
            continuous_imputer_instance, name_continuous_imputer,
            model_instance, name_model,
            separate_imputers=True  # Or however you want to specify
        )

    except Exception as e:  

        print(e)
    
        params = {
        "ordinal_imputer": name_ordinal_imputer, 
        "continuous_imputer": name_continuous_imputer, 
        "model": name_model, "train_shape" : df_X_train.shape, 
        "test_shape": df_X_test.shape
    }
        dict_results = {
        "params": params, 
        "imputation_time": None,
        "fitting_time": None, 
        "results_adj": None, 
        "results_org": None
    }
        
    # Optionally keep the all_dict_results list updated
    all_dict_results.append(dict_results)

    # Save the updated results back to the pickle file
    with open(results_file, 'wb') as f:
        pickle.dump(all_dict_results, f)

Using separate imputers for ordinal and continuous data.
No NaN in train data -> Keep as it is. 
No NaN in test data -> Keep as it is. 
Using separate imputers for ordinal and continuous data.
No NaN in train data -> Keep as it is. 
No NaN in test data -> Keep as it is. 
Using separate imputers for ordinal and continuous data.
No NaN in train data -> Keep as it is. 
No NaN in test data -> Keep as it is. 
Using separate imputers for ordinal and continuous data.
No NaN in train data -> Keep as it is. 
No NaN in test data -> Keep as it is. 
Using separate imputers for ordinal and continuous data.
No NaN in train data -> Keep as it is. 
No NaN in test data -> Keep as it is. 
Using separate imputers for ordinal and continuous data.
No NaN in train data -> Keep as it is. 
No NaN in test data -> Keep as it is. 
Using separate imputers for ordinal and continuous data.
No NaN in train data -> Keep as it is. 
No NaN in test data -> Keep as it is. 


KeyboardInterrupt: 

In [ ]:
# Store data (serialize)
with open(results_file, 'wb') as handle:
    pickle.dump(all_dict_results, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Print Table for reporting

In [5]:
results_file = "../pickle/training_2_dict_results.pickle"

In [7]:
with open(results_file, "rb") as input_file:
    all_dict_results = pickle.load(input_file)

In [8]:
pd.DataFrame(all_dict_results)

,params,imputation_time,fitting_time,results_adj,results_org
0,{'ordinal_imputer': 'SimpleImputer_most_freque...,3.246639,0.229391,"{'mse_score': [0.659831758923781, 0.4897431123...","{'mse_score': [0.6598317562332855, 0.489743104..."
1,{'ordinal_imputer': 'SimpleImputer_most_freque...,3.283444,0.098224,"{'mse_score': [1.1801224802168766, 0.950415349...","{'mse_score': [1.1801224892206006, 0.950415348..."
2,{'ordinal_imputer': 'SimpleImputer_most_freque...,3.266497,1.492615,"{'mse_score': [0.6679134194781107, 0.478566173...","{'mse_score': [0.6679134155298025, 0.478566167..."
3,{'ordinal_imputer': 'SimpleImputer_most_freque...,3.283154,0.077769,"{'mse_score': [1.3936376654364884, 1.040563386...","{'mse_score': [1.39363767778841, 1.04056338628..."
4,{'ordinal_imputer': 'SimpleImputer_most_freque...,3.311443,1.492246,"{'mse_score': [0.6555347035833681, 0.476021533...","{'mse_score': [0.655534699980677, 0.4760215269..."
...,...,...,...,...,...
86,"{'ordinal_imputer': 'NoImputer', 'continuous_i...",NaN,0.051667,"{'mse_score': [1.081409082021159, 0.7876511651...","{'mse_score': [1.0814090864925796, 0.787651164..."
87,"{'ordinal_imputer': 'NoImputer', 'continuous_i...",NaN,15.518824,"{'mse_score': [1.0560083392333595, 0.754741883...","{'mse_score': [1.0560083442605621, 0.754741880..."
88,"{'ordinal_imputer': 'NoImputer', 'continuous_i...",NaN,1.730847,"{'mse_score': [1.7491323070355045, 0.913167428...","{'mse_score': [1.7491323190951489, 0.913167424..."
89,"{'ordinal_imputer': 'NoImputer', 'continuous_i...",NaN,0.878380,"{'mse_score': [2.0471137028497073, 1.676544780...","{'mse_score': [2.0471137196295768, 1.676544782..."


In [9]:
def generate_metric_table(
    results_list,
    targets,
    metric_name,
    source="Adjusted",
    float_format="%.3f",
    csv_filename=None,
    sort_order="ascending"
):
    """
    Create a LaTeX and CSV table for a single metric across targets, models, and imputers,
    including mean ± std for performance, imputation time, and fitting time.

    Parameters
    ----------
    results_list : list of dict
        List of experiment results.
    targets : list of str
        Target names (e.g., ['ADNI_MEM', 'ADNI_EF', 'ADNI_VS', 'ADNI_LAN']).
    metric_name : str
        Metric to extract (e.g., 'mae_score').
    source : str
        'Adjusted' or 'Original'.
    float_format : str
        Format for floats (e.g., '%.3f').
    csv_filename : str or None
        If provided, saves the table to CSV.
    sort_order : str
        'ascending' or 'descending' for sorting by mean.

    Returns
    -------
    df : pd.DataFrame
        Final formatted DataFrame.
    latex_table : str
        LaTeX-formatted table string.
    """
    rows = []
    version_key = "results_adj" if source.lower() == "adjusted" else "results_org"

    for res in results_list:
        result_block = res.get(version_key)
        if result_block is None:
            continue

        metric_values = result_block.get(metric_name)
        if metric_values is None:
            continue

        if len(metric_values) != len(targets):
            continue

        ordinal_imputer = res["params"].get("ordinal_imputer")
        continuous_imputer = res["params"].get("continuous_imputer")
        model = res["params"].get("model")

        values = np.array(metric_values, dtype=np.float64)
        mean_val = np.mean(values)
        std_val = np.std(values)

        # Time metrics
        imp_times = np.array(res.get("imputation_time", []), dtype=np.float64)
        fit_times = np.array(res.get("fitting_time", []), dtype=np.float64)

        row = {
            "Ordinal Imputer": ordinal_imputer,
            "Continuous Imputer": continuous_imputer,
            "Model": model,
            "Mean": mean_val,
            "Mean ± SD": f"{mean_val:.3f} ± {std_val:.3f}",
            "Imputation Time": f"{imp_times.mean():.2f}" if imp_times.size > 0 else "N/A",
            "Fitting Time": f"{fit_times.mean():.2f}" if fit_times.size > 0 else "N/A"
        }

        row.update({target: val for target, val in zip(targets, values)})
        rows.append(row)

    df = pd.DataFrame(rows)

    # Reorder columns for display
    display_cols = (
        ["Ordinal Imputer", "Continuous Imputer", "Model"] +
        targets +
        ["Mean ± SD", "Imputation Time", "Fitting Time"]
    )
    df = df.sort_values(by="Mean", ascending=(sort_order == "ascending"))
    df = df[display_cols]

    df.drop_duplicates(subset=["Ordinal Imputer", "Continuous Imputer", "Model"] +
        targets +
        ["Mean ± SD",], inplace=True)

    # Save CSV if requested
    if csv_filename:
        df.to_csv(csv_filename, index=False)

    # Generate LaTeX table
    latex_table = df.to_latex(
        index=False,
        escape=False,
        float_format=float_format,
        caption=f"{metric_name.replace('_', ' ').upper()} across targets with timing info",
        label=f"tab:{metric_name}",
        longtable=False
    )

    return df, latex_table


In [17]:
latex_df, latex_mae = generate_metric_table(
    results_list=all_dict_results,
    targets=['ADNI_MEM', 'ADNI_EF', 'ADNI_VS', 'ADNI_LAN'],
    metric_name='corr',
    source="Adjusted",
    csv_filename="../tables/2_training_train_test_corr_adjusted_sorted.csv",
    sort_order="descending"
)
print(latex_mae)

\begin{table}
\caption{CORR across targets with timing info}
\label{tab:corr}
\begin{tabular}{lllrrrrlll}
\toprule
Ordinal Imputer & Continuous Imputer & Model & ADNI_MEM & ADNI_EF & ADNI_VS & ADNI_LAN & Mean ± SD & Imputation Time & Fitting Time \\
\midrule
SimpleImputer_most_frequent & KNNImputer & TabNetRegressor_default & 0.773 & 0.676 & 0.581 & 0.819 & 0.712 ± 0.092 & 3.38 & 13.77 \\
SimpleImputer_most_frequent & KNNImputer & MultiTaskLasso_tuned & 0.619 & 0.707 & 0.377 & 0.675 & 0.594 ± 0.130 & 3.31 & 1.49 \\
SimpleImputer_most_frequent & KNNImputer & LinearRegression & 0.606 & 0.692 & 0.402 & 0.666 & 0.592 ± 0.114 & 3.25 & 0.23 \\
SimpleImputer_most_frequent & KNNImputer & MultiTaskElasticNet_tuned & 0.608 & 0.705 & 0.382 & 0.669 & 0.591 ± 0.126 & 3.27 & 1.49 \\
SimpleImputer_most_frequent & KNNImputer & RandomForestRegressor & 0.635 & 0.677 & 0.343 & 0.627 & 0.570 ± 0.133 & 3.29 & 51.80 \\
NoImputer & NoImputer & XGBoostRegressor_tuned & 0.481 & 0.591 & 0.307 & 0.656 & 0.509 ± 

In [11]:
latex_df, latex_mae = generate_metric_table(
    results_list=all_dict_results,
    targets=['ADNI_MEM', 'ADNI_EF', 'ADNI_VS', 'ADNI_LAN'],
    metric_name='r2',
    source="Adjusted",
    csv_filename="../tables/2_training_train_test_r2_adjusted_sorted.csv",
    sort_order="descending"
)
print(latex_mae)

\begin{table}
\caption{R2 across targets with timing info}
\label{tab:r2}
\begin{tabular}{lllrrrrlll}
\toprule
Ordinal Imputer & Continuous Imputer & Model & ADNI_MEM & ADNI_EF & ADNI_VS & ADNI_LAN & Mean ± SD & Imputation Time & Fitting Time \\
\midrule
SimpleImputer_most_frequent & KNNImputer & LinearRegression & 0.333 & 0.444 & 0.145 & 0.310 & 0.308 ± 0.107 & 3.25 & 0.23 \\
SimpleImputer_most_frequent & KNNImputer & MultiTaskLasso_tuned & 0.337 & 0.460 & 0.125 & 0.291 & 0.303 ± 0.120 & 3.31 & 1.49 \\
SimpleImputer_most_frequent & KNNImputer & TabNetRegressor_default & 0.375 & 0.306 & 0.230 & 0.297 & 0.302 ± 0.051 & 3.38 & 13.77 \\
SimpleImputer_most_frequent & KNNImputer & MultiTaskElasticNet_tuned & 0.325 & 0.457 & 0.135 & 0.279 & 0.299 ± 0.115 & 3.27 & 1.49 \\
SimpleImputer_most_frequent & KNNImputer & RandomForestRegressor & 0.179 & 0.368 & 0.114 & 0.072 & 0.183 ± 0.113 & 3.29 & 51.80 \\
NoImputer & NoImputer & XGBoostRegressor_tuned & -0.026 & 0.275 & 0.032 & 0.139 & 0.105 ± 0.1

In [15]:
latex_df, latex_mse = generate_metric_table(
    results_list=all_dict_results,
    targets=['ADNI_MEM', 'ADNI_EF', 'ADNI_VS', 'ADNI_LAN'],
    metric_name='mse_score',
    source="Adjusted",
    csv_filename="../tables/2_training_train_test_mse_adjusted_sorted.csv",
    sort_order="ascending"
)
print(latex_mse)

\begin{table}
\caption{MSE SCORE across targets with timing info}
\label{tab:mse_score}
\begin{tabular}{lllrrrrlll}
\toprule
Ordinal Imputer & Continuous Imputer & Model & ADNI_MEM & ADNI_EF & ADNI_VS & ADNI_LAN & Mean ± SD & Imputation Time & Fitting Time \\
\midrule
SimpleImputer_most_frequent & KNNImputer & LinearRegression & 0.660 & 0.490 & 0.410 & 0.567 & 0.531 ± 0.093 & 3.25 & 0.23 \\
SimpleImputer_most_frequent & KNNImputer & MultiTaskLasso_tuned & 0.656 & 0.476 & 0.419 & 0.582 & 0.533 ± 0.092 & 3.31 & 1.49 \\
SimpleImputer_most_frequent & KNNImputer & MultiTaskElasticNet_tuned & 0.668 & 0.479 & 0.414 & 0.593 & 0.538 ± 0.098 & 3.27 & 1.49 \\
SimpleImputer_most_frequent & KNNImputer & TabNetRegressor_default & 0.618 & 0.612 & 0.369 & 0.578 & 0.544 ± 0.102 & 3.38 & 13.77 \\
SimpleImputer_most_frequent & KNNImputer & RandomForestRegressor & 0.812 & 0.557 & 0.424 & 0.763 & 0.639 ± 0.156 & 3.29 & 51.80 \\
NoImputer & NoImputer & XGBoostRegressor & 0.997 & 0.553 & 0.498 & 0.746 & 0.69

In [13]:
latex_df, latex_mae = generate_metric_table(
    results_list=all_dict_results,
    targets=['ADNI_MEM', 'ADNI_EF', 'ADNI_VS', 'ADNI_LAN'],
    metric_name='mae_score',
    source="Adjusted",
    csv_filename="../tables/2_training_train_test_mae_adjusted_sorted.csv",
    sort_order="ascending"
)
print(latex_mae)

\begin{table}
\caption{MAE SCORE across targets with timing info}
\label{tab:mae_score}
\begin{tabular}{lllrrrrlll}
\toprule
Ordinal Imputer & Continuous Imputer & Model & ADNI_MEM & ADNI_EF & ADNI_VS & ADNI_LAN & Mean ± SD & Imputation Time & Fitting Time \\
\midrule
SimpleImputer_most_frequent & KNNImputer & MultiTaskLasso_tuned & 0.622 & 0.551 & 0.578 & 0.595 & 0.586 ± 0.026 & 3.31 & 1.49 \\
SimpleImputer_most_frequent & KNNImputer & LinearRegression & 0.640 & 0.551 & 0.565 & 0.600 & 0.589 ± 0.034 & 3.25 & 0.23 \\
SimpleImputer_most_frequent & KNNImputer & MultiTaskElasticNet_tuned & 0.630 & 0.554 & 0.580 & 0.600 & 0.591 ± 0.028 & 3.27 & 1.49 \\
SimpleImputer_most_frequent & KNNImputer & TabNetRegressor_default & 0.691 & 0.608 & 0.536 & 0.595 & 0.607 ± 0.055 & 3.38 & 13.77 \\
SimpleImputer_most_frequent & KNNImputer & RandomForestRegressor & 0.669 & 0.640 & 0.608 & 0.648 & 0.641 ± 0.022 & 3.29 & 51.80 \\
NoImputer & NoImputer & XGBoostRegressor & 0.754 & 0.635 & 0.643 & 0.644 & 0.66

In [14]:
latex_df.Model.value_counts()

Model
RandomForestRegressor                  5
GatedAdditiveTreeEnsembleConfig_tab    3
TabTransformerConfig_tab               3
MultiTaskLasso_tuned                   2
LinearRegression                       2
MultiTaskElasticNet_tuned              2
TabNetRegressor_default                2
TabNetRegressor_custom                 2
PLSRegression_4_components             2
MultiTaskElasticNet                    2
DANetConfig_tab                        2
MultiTaskLasso                         2
TabNetModelConfig_tab                  2
XGBoostRegressor                       1
XGBoostRegressor_tuned                 1
Name: count, dtype: int64